In [92]:
import pandas as pd
import sqlite3
import csv
import seaborn as sns

### Creating a connection and database (Regular SQL method)

In [93]:
conn = sqlite3.connect('Databankdraft')
c = conn.cursor()

### Importing and loading datasets with pandas

In [94]:
df_sales = pd.read_csv ('Sales.csv')
df_menu = pd.read_csv ('menu.csv')
df_mem = pd.read_csv ('members.csv')

### Clean space in column headers

In [95]:
df_sales.columns

Index(['   customer_id', ' order_date', ' product_id'], dtype='object')

In [120]:
df_sales.columns = df_sales.columns.str.strip()

In [121]:
df_sales.columns

Index(['customer_id', 'order_date', 'product_id'], dtype='object')

In [123]:
df_menu.columns

Index(['product_id', 'product_name', 'price'], dtype='object')

In [124]:
df_menu.columns = df_menu.columns.str.strip()

In [126]:
df_menu.columns

Index(['product_id', 'product_name', 'price'], dtype='object')

In [128]:
df_mem.columns = df_mem.columns.str.strip()

In [129]:
df_mem.columns

Index(['customer_id', 'join_date'], dtype='object')

### Inserting values into a table

In [130]:
df_sales.to_sql('df_sales', conn,if_exists='replace',index=False)
df_menu.to_sql('df_menu', conn,if_exists='replace',index=False)
df_mem.to_sql('df_mem', conn,if_exists='replace',index=False)

### Verifying Data types

In [133]:
qry = """PRAGMA table_info(df_sales)"""
pd.read_sql_query(qry, con=conn)

,cid,name,type,notnull,dflt_value,pk
0,0,customer_id,TEXT,0,None,0
1,1,order_date,TEXT,0,None,0
2,2,product_id,INTEGER,0,None,0


In [134]:
qry = """PRAGMA table_info(df_menu)"""
pd.read_sql_query(qry, con=conn)

,cid,name,type,notnull,dflt_value,pk
0,0,product_id,INTEGER,0,None,0
1,1,product_name,TEXT,0,None,0
2,2,price,INTEGER,0,None,0


In [135]:
qry = """PRAGMA table_info(df_mem)"""
pd.read_sql_query(qry, con=conn)

,cid,name,type,notnull,dflt_value,pk
0,0,customer_id,TEXT,0,None,0
1,1,join_date,TEXT,0,None,0


In [138]:
qry = """SELECT
  	product_id,
   product_name,
   price
FROM df_menu
ORDER BY price DESC;"""

pd.read_sql_query(qry, con=conn)

,product_id,product_name,price
0,2,curry,15
1,3,ramen,12
2,1,sushi,10


In [139]:
qry = """SELECT 
df_sales.customer_id, 
df_menu.price, 
df_sales.order_date
FROM df_sales
INNER JOIN df_menu ON df_sales.product_id = df_menu.product_id;
"""
pd.read_sql_query(qry, con=conn)

,customer_id,price,order_date
0,A,10,2021-01-01
1,A,15,2021-01-01
2,A,15,2021-01-07
3,A,12,2021-01-10
4,A,12,2021-01-11
5,A,12,2021-01-11
6,B,15,2021-01-01
7,B,15,2021-01-02
8,B,10,2021-01-04
9,B,10,2021-01-11


1. What is the total amount each customer spent at the restaurant?

In [141]:
qry = """SELECT
SUM(price)
FROM df_menu
"""

pd.read_sql_query(qry, con=conn)

,SUM(price)
0,37
